<p><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"></p>
<h1 id="unSkript-Runbooks-&para;">unSkript Runbooks <a class="jp-InternalAnchorLink" href="#-unSkript-Runbooks-" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#unSkript-Runbooks-&para;" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success"><strong>&nbsp;This runbook updates the expiration tag for AWS Resources that are about to expire.</strong></div>
<p>&nbsp;</p>
<center><center>
<h2 id="Update-AWS-Resources-about-to-expire&para;">Update AWS Resources about to expire<a class="jp-InternalAnchorLink" href="#Enforce-Mandatory-Tags-Across-All-AWS-Resources" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Update-AWS-Resources-about-to-expire&para;" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Steps-Overview&para;" target="_self">&para;</a></h1>
<ol>
<li>Get all resources in a given region with an expiratiuon tag.</li>
<li>Filter those that have expired or are about to expire</li>
<li>Update manually with the date picker.</li>
<li>Update the epiration tag to the selected AWS Resources.</li>
<li>Send a Slack message on number of expiring resources.</li>
</ol>
<p>&nbsp;</p>
<p>If this RunBook is run Programatically - no dates will be changed - just a Slack alert sent.</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
</center>

<p>Here we will use unSkript AWS Get Resources with the expiration tag&nbsp; - the tag name is an input parameter for the runbook.</p>

In [12]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_get_resources_with_expiration_tag_printer(output):
    if output is None:
        return
    pprint.pprint(f"there are {len(output)} resources missing tag {Tag_Key}. We can fix a max of 20." )


@beartype
def aws_get_resources_with_expiration_tag(handle, region: str, tag:str) -> List:

    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = []
    try:
        response = aws_get_paginator(ec2Client, "get_resources", "ResourceTagMappingList")
        for resources in response:
            if resources["Tags"]:
                #has tags
                tags = resources['Tags']
                for kv in resources['Tags']:
                    if kv["Key"] == tag:
                        #we have found an expiration tag
                        temp ={'arn': [resources["ResourceARN"]], 'expires':kv["Value"]}
                        print(temp)
                        result.append(temp)

    except Exception as error:
        result.append({"error":error})

    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "tag": "expiration_tag"
    }''')
task.configure(outputName="ExpirationResources")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_resources_with_expiration_tag, lego_printer=aws_get_resources_with_expiration_tag_printer, hdl=hdl, args=args)

<p>Now, we filter for ony resrouces that have expired, or are bout to expire, and display in list with a date picker.</p>
<p>&nbsp;</p>
<p>Updating the date will allow us to change the value to the expiration tag.</p>
<p>&nbsp;</p>

In [4]:
from datetime import datetime, timedelta

#print(ExpirationResources)
expiringList = []
# Get the current date
current_date = datetime.now()

# Calculate the date days_to_expire days from now
future_date = current_date + timedelta(days=days_to_expire)

for resource in ExpirationResources:
    expires = datetime.strptime(resource['expires'], "%m/%d/%Y")
    if expires < future_date:
        expiringList.append(resource)

print(expiringList)

In [5]:
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display

# Sample list of dictionaries


def update_expiration_date(expiration_date, arn):
    for item in expiringList:
        if item["arn"][0] == arn:
            item["expires"] = expiration_date.strftime("%m/%d/%Y")
    print(expiringList)

def on_date_change(change):
    arn = change.owner.description
    expiration_date = change.new
    update_expiration_date(expiration_date, arn)

# Create a date picker for each ARN
for item in expiringList:
    expiration_date = datetime.strptime(item["expires"], "%m/%d/%Y").date()
    date_picker = widgets.DatePicker(description=item["arn"][0], 
                                     style=dict(description_width='initial'),
                                     layout=dict(width='80%'),
                                     value=expiration_date)
    date_picker.observe(on_date_change, names='value')
    display(date_picker)


In [5]:
print(expiringList)

<p>Here we will use unSkript AWS Attach Tags to Resources Lego.&nbsp;</p>
<p>&nbsp;</p>
<p>The updated dates from the date picker will be used to replace the current value in the expiration tag.</p>

In [32]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Dict
from pydantic import BaseModel, Field

from beartype import beartype
@beartype
def aws_attach_tags_to_resources_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_attach_tags_to_resources(
    handle,
    resource_arn: list,
    tag_key: str,
    tag_value: str,
    region: str
    ) -> Dict:
    """aws_attach_tags_to_resources Returns an Dict of resource info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type resource_arn: list
        :param resource_arn: Resource ARNs.

        :type tag_key: str
        :param tag_key: Resource Tag Key.

        :type tag_value: str
        :param tag_value: Resource Tag value.

        :type region: str
        :param region: Region to filter resources.

        :rtype: Dict of resource info.
    """
    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = {}
    try:
        response = ec2Client.tag_resources(
            ResourceARNList=resource_arn,
            Tags={tag_key: tag_value}
            )
        result = response

    except Exception as error:
        result["error"] = error

    return result



task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "region": "Region",
    "resource_arn": "iter.get(\\"arn\\")",
    "tag_key": "expiration_tag",
    "tag_value": "iter.get(\\"expires\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "[x for x in expiringList]",
    "iter_parameter": ["resource_arn","tag_value"]
    }''')
task.configure(outputName="test")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_attach_tags_to_resources, lego_printer=aws_attach_tags_to_resources_printer, hdl=hdl, args=args)

In [9]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint
from pydantic import BaseModel, Field
from beartype import beartype
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)

from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfully Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found') from e
        if e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid') from e
        if e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in') from e
        if e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived') from e
        if e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long') from e
        if e.response['error'] == 'no_text':
            raise Exception('Message text was not provided') from e
        if e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting') from e
        if e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel') from e
        if e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace') from e
        if e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided') from e
        if e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied') from e
        if e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied') from e
        if e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user') from e
        if e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked') from e
        if e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message') from e
        if e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later') from e
        if e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable') from e
        if e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message') from e
        if e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue') from e
        if e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated') from e
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}') from e

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {str(e)}")
        return f"Unable to send message on {channel}"



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "\\"devrel_doug_test1\\"",
    "message": "f\\"There are {len(expiringList)} AWS resources set to expire in the next {days_to_expire} days! Use the AWS Resources About To Expire RunBook to manually update these dates to avoid any deletion of important resources\\""
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to attach tags. This Runbook gets the list of all untagged resources of a given region, discovers tag keys of the given region and attaches mandatory tags to all the untagged resource. To view the full platform capabilities of unSkript please visit https://unskript.com